In [1]:
from bs4 import BeautifulSoup as bs
from IPython.display import display
import requests
import pandas as pd
import io
import time

In [2]:
def fetch(url):
    headers={'Authority':'xxxx','Method':'GET','Path':'/xxxx','Scheme':'https','Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9','Connection': 'keep-alive','Content-Type': 'text/plain;charset=UTF-8','Cache-Control':'max-age=0','Upgrade-Insecure-Requests': '1','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36','x-client-data': 'CJG2yQEIpbbJAQjEtskBCKmdygEI2J3KAQioo8oBCIKkygE='}
    page=requests.Session().get(url,headers=headers)
    return bs(page.text,'lxml')

In [3]:
def getProductsData(soup):
## creates list of products for a single page    
    productsInAPage=soup.find('div',attrs={'class':'a-row s-result-list-parent-container'}).find('ul','s-result-list s-col-3 s-col-ws-4 s-result-list-hgrid s-height-equalized s-grid-view s-text-condensed').find_all('li')
    temporaryList=[]
    for i in productsInAPage:
        temporaryList.append(i)
    
    return(temporaryList)


In [4]:
def choice(options,optionLinks):
    print('Enter Option No.:')
    for i in range(len(options)):
        print(i+1,options[i])
    noOfOptions=len(optionLinks)
    while True:
        userChoice=input()
        if userChoice.isdigit():
            userChoice=int(userChoice)
            if userChoice in range(1,noOfOptions+1):
                break
        print(f'Enter correct option which is a number between 1 and {noOfOptions}\n')
    print('\n\n')
    return(optionLinks[userChoice-1])

In [5]:
def expanding():
    
    optionLinks=['https://www.amazon.in/b/ref=s9_acss_bw_cg_GrSBCNvP_2a1_w?_encoding=UTF8&ie=UTF8&node=4859478031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-4&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
    ,'https://www.amazon.in/s/ref=s9_acss_bw_cg_GrSBCNvP_2b1_w?rh=i%3Agrocery%2Cn%3A4859479031&ie=UTF8&bbn=2454179031&pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-11&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
    ,'https://www.amazon.in/b/ref=s9_acss_bw_cg_GrSBCNvP_2c1_w?_encoding=UTF8&ie=UTF8&node=4859498031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-4&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
    ,'https://www.amazon.in/b/ref=s9_acss_bw_cg_GrSBCNvP_8a1_w?_encoding=UTF8&ie=UTF8&node=4859499031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-4&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
    ,'https://www.amazon.in/b/ref=s9_acss_bw_cg_GrSBCNvP_8b1_w?_encoding=UTF8&ie=UTF8&node=14838817031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-4&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
    ,'https://www.amazon.in/b/ref=s9_acss_bw_cg_GrSBCNvP_8c1_w?_encoding=UTF8&ie=UTF8&node=10405401031&pf_rd_m=A1VBAL9TL5WCBF&pf_rd_s=merchandised-search-4&pf_rd_r=57N6R66ADBR0F618BZ89&pf_rd_t=101&pf_rd_p=e165784a-1d1e-5d35-9c2c-726cf0cefc59&pf_rd_i=2454178031'
             ]
    options=['Bevrages','Cooking and Baking supplies','Snacks','Chocolates and Desserts','Packaged foods','Breakfast foods']
    
    baseUrl=choice(options,optionLinks)
    while True:
        soup=fetch(baseUrl)
        
        subMenu=soup.find('div','a-row a-expander-container a-expander-extend-container')
        if len(subMenu)>0:
            options=[]
            optionLinks=[]
            for i in subMenu:
                menuLink=i.find('a')
                options.append(menuLink.text)
                optionLinks.append('https://www.amazon.in/'+menuLink.attrs.get('href'))
            baseUrl=choice(options,optionLinks)
        else:
            break
    return baseUrl

In [6]:
def getNextPage(address):
    soup=fetch(address)
    if soup.find('a','pagnNext')!=None:
        return 'https://www.amazon.in'+soup.find('a','pagnNext')['href']

In [7]:
def numberOfPages(address):
    soup=fetch(address)
    if soup.find('span','pagnDisabled'):
        maxPage=int(soup.find('span','pagnDisabled').text)
    elif soup.find_all('span','pagnLink'):
        allPage=soup.find_all('span','pagnLink')
        maxPage=int(allPage[len(allPage)-1].text)
    else:
        maxPage=1
    print(f'Till which page do you want data[Maximum number of pages are {maxPage}].')
    while True:
        noPages=input()
        if noPages.isdigit():
            noPages=int(noPages)
            if noPages in range(1,maxPage+1):
                break
        print(f'Enter correct option which is a number between 1 and {maxPage}\n')
    return noPages

In [8]:
def priceExtractor(w):
    temporaryList=[]
    for i in w.find_all('div','a-row a-spacing-mini'):
        temp1=i.find_all('div','a-row a-spacing-none')
        if temp1:
            for j in temp1:
                temporaryList.append(j)
    theCorrectPrice=[]
## using temp1 and temp2 again for new data    
    for i in temporaryList:
        temp1=i.find_all('a','a-link-normal a-text-normal')
        if temp1==[]:
            temp2=i.find_all('a','a-size-small a-link-normal a-text-normal') 
            if temp2:
                theCorrectPrice.append(temp2)
        else:
            theCorrectPrice.append(temp1)
    for i in theCorrectPrice:
        for j in i:
            return(j)

In [9]:
def extractor(data_list=[]):
    data=[]
    for li in data_list:
        if(li.find('h2',attrs={'class':'a-size-base s-inline s-access-title a-text-normal'})!=None):
            heading=li.find('h2',attrs={'class':'a-size-base s-inline s-access-title a-text-normal'})
            link=li.find('a',attrs={'class':'a-link-normal a-text-normal'})
            if(priceExtractor(li)!=None):
                price=priceExtractor(li)
            else:
                price='NaN'
            if(li.find('span',attrs={'class':'a-size-small a-color-price'})):
                discount=li.find('span',attrs={'class':'a-size-small a-color-price'})
            else:
                discount=None
            if(li.find('i',attrs={'class':'a-icon a-icon-star a-star-5'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-5'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-4-5'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-4-5'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-4'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-4'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-3-5'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-3-5'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-3'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-3'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-2'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-2'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-2'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-2-5'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-1-5'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-1-5'}).find('span',attrs={'class':'a-icon-alt'}).text
            elif(li.find('i',attrs={'class':'a-icon a-icon-star a-star-1'})!=None):
                ratings=li.find('i',attrs={'class':'a-icon a-icon-star a-star-1'}).find('span',attrs={'class':'a-icon-alt'}).text
            else:
                ratings=None
            if(li.find('img',attrs={'class':'s-access-image cfMarker'})!=None):
                imgsrc=li.find('img',attrs={'class':'s-access-image cfMarker'})
            else:
                imgsrc='Not available'
            if(li.find_all('span',attrs={'class':'a-size-small a-color-secondary'})!=[]):
                company=li.find_all('span',attrs={'class':'a-size-small a-color-secondary'})
            else:
                company='NaN'
            item={}
            if(heading.text.endswith('[Sponsored]',0,11)):
                item['Heading']=heading.text[11:]
                item['Link']="https://www.amazon.in"+link.attrs.get('href')
            else:
                item['Heading']=heading.text
                item['Link']=link.attrs.get('href')
            if price!='NaN':
                item['Price']='₹'+(price.text).replace(u'\xa0', u'')
                if len(item['Price'])>14:
                    if item['Price'][-6:-1]=='offer':
                        item['Price']=item['Price'][:-14]
                    elif item['Price'][-7:-1]=='offers':
                        item['Price']=item['Price'][:-15]
            else:
                item['Price']=price
            if(discount):
                item['Discount']=(discount.text).replace(u'\xa0', u'').split()
                item['Discount']='₹'+"".join(item['Discount'])
            else:
                item['Discount']="No discount available"
            item['Ratings']=ratings
            if(company!='NaN'):
                item['Company']=''.join(company[0].text+company[1].text)
                if item['Company'][:3]=='by ':
                    item['Company']=item['Company'][3:]
            else:
                item['Company']=company
            item['Imgsrc']=imgsrc.attrs.get('src')
            data.append(item)
        else:
            pass
       
        
        
    return data

In [10]:
def utilityFunction(temporaryList):
    finalList=[]
    for i in range(len(temporaryList)):
        finalList=finalList+[extractor(temporaryList[i])]
    return(finalList)

In [11]:
def forPandas(givenList):
    tempList=[]
    dataForPandas=[]
    for i in givenList:
        for j in i:
            tempList.append(i)
    for i in tempList:
        dataForPandas.append(*i)
    return(dataForPandas)

In [12]:
def acceptsPage():
    dataEx=[]
    address=expanding()
    pages=numberOfPages(address)
    for _ in range(pages):
        soup=fetch(address)
        tempDataEx=utilityFunction(getProductsData(soup))
        for i in tempDataEx:
            dataEx.append(i) 
        address=getNextPage(address)   
    dataToUse=forPandas(dataEx)
    saveValue(dataToUse)
    displayedData=pd.DataFrame(dataToUse)
    return(displayedData)
    

In [13]:
def saveValue(dataToUse):
    with io.open("amazondata.txt",'w',encoding='utf-8') as file:
        fileData=file.write("\n----***----\n")
        for item in dataToUse:
            for key,keyValue in item.items():
                fileData=file.write(str(key)+'-->>'+str(keyValue)+'\n')
            fileData=file.write("\n----***----\n")      

In [20]:
if __name__ == "__main__":
    try:
        displayData=acceptsPage()
    except:
        print('Please try again')
    try:
        display(displayData)
    except:
        pass

Enter Option No.:
1 Bevrages
2 Cooking and Baking supplies
3 Snacks
4 Chocolates and Desserts
5 Packaged foods
6 Breakfast foods


 1





Enter Option No.:
1 Coffee & Espresso
2 Cola & Soft Drinks
3 Concentrated Syrups & Squash
4 Fruit Juice
5 Hot Chocolate & Malted Drinks
6 Iced Tea & Lemonade
7 Powdered Drink Mixes
8 Tea


 1





Till which page do you want data[Maximum number of pages are 78].


 6


,Company,Discount,Heading,Imgsrc,Link,Price,Ratings
0,Bru,₹12(4%),"Bru Gold Instant Coffee, 100g",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/gp/slredirect/picassoRed...,₹233,4.2 out of 5 stars
1,Blue Tokai Coffee Roasters,No discount available,Blue Tokai Coffee Roasters Vienna Roast (Dark ...,https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/gp/slredirect/picassoRed...,₹400,4.2 out of 5 stars
2,Colombian Brew Coffee,₹1,"Colombian Brew 100% Arabica Filter Coffee, Roa...",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/gp/slredirect/picassoRed...,₹298,3.9 out of 5 stars
3,Bru,₹12(4%),"Bru Gold Instant Coffee, 100g",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Bru-Gold-Instant-Coffee-...,₹233,4.2 out of 5 stars
4,Nescafé,₹17(6%),"Nescafé Classic Coffee, 100g Glass Jar",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Nescaf%C3%A9-Classic-Cof...,₹248,4.3 out of 5 stars
5,Coffee Day,₹39(20%),"Coffee Day Ultra Rich, 500g",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Coffee-Day-Ultra-Rich-50...,₹153,3.6 out of 5 stars
6,Bru,₹9(5%),"BRU Instant Coffee, 100g",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Bru-BRU-Instant-Coffee-1...,₹156,4.1 out of 5 stars
7,Bru,₹10(11%),"Bru Instant Coffee, 50g",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Bru-Instant-Coffee-50g/d...,₹80,3.8 out of 5 stars
8,Nescafé,₹50(11%),"Nescafe Gold Blend Instant Coffee Powder, 100g...",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Nescafe-Blend-Instant-Co...,₹370,4.4 out of 5 stars
9,Nescafé,₹23(5%),"Nescafé Classic Coffee, 200g Stabilo Pack",https://images-eu.ssl-images-amazon.com/images...,https://www.amazon.in/Nescaf%C3%A9-Classic-Cof...,₹417,4.2 out of 5 stars
